# Inferential Analysis on Changes in Snow Depth Due to Elevation

## Introduction

During the winter season, many people like to go hit the slopes to have some fun, and even more so in BC as the province is home to many world-renowned skiing destinations. When choosing a location, one of the main factors perceived by many that contribute to a greater skiing experience is the elevation of the mountain, as it’s believed that a higher elevation leads to an increase in snow depth which helps improve one’s skiing experience. This relationship between elevation and snow depth is a popular claim by the public, with one study suggesting a positive correlation between elevation and snow depth and another suggesting a similar proposition up to an elevation of 3300 meters (Grünewald et al., 2014; Kirchner et al., 2014). As skiing hobbyists ourselves, we wanted to set out and investigate whether this claim is statistically significant. We decided to focus on locations within BC for relevancy, and settled on comparing the snow depths captured by automated snow weather stations placed in two locations with a distinct difference in elevation: Callaghan, an area just west of Whistler with an elevation of 1017 meters, and Silver Star Mountain, home to the famous Silver Star Ski Resort with an elevation of 1839 meters. As such, our question will be: does the higher elevation of Silver Star Mountain have a greater snow depth than the lower elevation of Callaghan?

Our chosen location and scale parameter will be the mean and standard deviation, respectively. Finding the means of the samples allows us to carry out a one-tailed two-sample t-test which would be relevant for answering the question, while finding the standard deviations will allow us to observe the variability of the observations in each sample.

We’ll be using two datasets from the BC government containing samples of snow depth in centimeters, captured by snow weather stations located in Callaghan and Silver Star Mountain as previously stated. The datasets will be filtered down to contain observations only in the month of January from 2019-2022 for reduced variability in measurements caused by seasonal changes (Ye & Ellison, 2003).

## Preliminary Results

Loading necessary libraries:

In [48]:
library(tidyverse)
library(repr)
library(digest)
#library(infer)
library(gridExtra)
#library(cowplot)

# Install the package
install.packages("lubridate")
install.packages("infer")
# Load the package
library(lubridate)
library(infer)

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

also installing the dependency ‘patchwork’




Reading the csv data files from the data folder. Each corresponding csv file from each location is loaded into a different variable.

In [49]:
snow_depth_callaghan <- read.csv(file = 'data/Callaghan/SD.csv', col.names = c("Time", "NULL", "SnowDepth"))
snow_depth_silverstar <- read.csv(file = 'data/SilverStar/SD.csv', col.names = c("Time", "NULL", "SnowDepth"))
air_temp_callaghan <- read.csv(file = 'data/Callaghan/AT.csv', col.names = c("Time", "NULL", "AirTempCelcius"))
air_temp_silverstar <- read.csv(file = 'data/SilverStar/AT.csv', col.names = c("Time", "NULL", "AirTempCelcius"))

head(air_temp_silverstar)
head(snow_depth_silverstar)

,Time,NULL.,AirTempCelcius
,<chr>,<chr>,<chr>
1,"GRADEDESCRIPTIONS-3=GAP, -2=UNUSABLE, -1=UNSPECIFIED, 0=UNDEF, 11=POOR, 21=ESTIMATED, 25=BEST PRACTICE, 31=GOOD, 41=VERYGOOD, 51=EXCELLENT, APPROVALDESCRIPTIONS 800=WORKING, 900=IN REVIEW, 1200=APPROVED",,
2,Timestamp (UTC),Event Timestamp (UTC),Value (°C)
3,2018-09-10 18:00:00,,3
4,2018-09-10 19:00:00,,3.8
5,2018-09-10 20:00:00,,5.7
6,2018-09-10 21:00:00,,6.3


,Time,NULL.,SnowDepth
,<chr>,<chr>,<chr>
1,"GRADEDESCRIPTIONS-3=GAP, -2=UNUSABLE, -1=UNSPECIFIED, 0=UNDEF, 11=POOR, 21=ESTIMATED, 25=BEST PRACTICE, 31=GOOD, 41=VERYGOOD, 51=EXCELLENT, APPROVALDESCRIPTIONS 800=WORKING, 900=IN REVIEW, 1200=APPROVED",,
2,Timestamp (UTC),Event Timestamp (UTC),Value (cm)
3,2015-08-13 22:00:00,,0
4,2015-08-13 23:00:00,,5
5,2015-08-14 00:00:00,,-1
6,2015-08-14 01:00:00,,-4


As you can see above, we must from tidy and wrangle the data. 

In [50]:
# For Each DataSet we filter away the Null/NA values. Then we convert the charachter dates to R's built in date object type.
# Next for the snowdepth we filter away the negative snow depths (impossible/instrument error)
snow_depth_silverstar <- snow_depth_silverstar %>%
    filter(!is.na(SnowDepth)) %>%
    mutate(Time = strptime(Time, tz = "GMT", format = "%Y-%m-%d %H:%M:%S"),
          SnowDepth = as.numeric(SnowDepth),
          Location = "SilverStar") %>%
    filter(SnowDepth >= 0, !is.na(SnowDepth)) |>
    select(Time, SnowDepth, Location)

snow_depth_callaghan <- snow_depth_callaghan %>%
    filter(!is.na(SnowDepth)) %>%
    mutate(Time = strptime(Time, tz = "GMT", format = "%Y-%m-%d %H:%M:%S"),
          SnowDepth = as.numeric(SnowDepth),
          Location = "Callaghan") %>%
    filter(SnowDepth >= 0, !is.na(SnowDepth)) |>
    select(Time, SnowDepth, Location)

air_temp_callaghan <- air_temp_callaghan %>%
    filter(!is.na(AirTempCelcius)) %>%
    mutate(Time = strptime(Time, tz = "GMT", format = "%Y-%m-%d %H:%M:%S"),
          AirTempCelcius = as.numeric(AirTempCelcius),
          Location = "Callaghan") %>%
    filter(!is.na(AirTempCelcius)) |>
    select(Time, AirTempCelcius, Location)

air_temp_silverstar <- air_temp_silverstar %>%
    filter(!is.na(AirTempCelcius)) %>%
    mutate(Time = strptime(Time, tz = "GMT", format = "%Y-%m-%d %H:%M:%S"),
          AirTempCelcius = as.numeric(AirTempCelcius),
          Location = "SilverStar") %>%
    filter(!is.na(AirTempCelcius)) |>
    select(Time, AirTempCelcius, Location)



head(snow_depth_silverstar)
head(snow_depth_callaghan)
head(air_temp_silverstar)
head(air_temp_callaghan)

Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”
Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”
Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”
Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”


,Time,SnowDepth,Location
,<dttm>,<dbl>,<chr>
1,2015-08-13 22:00:00,0,SilverStar
2,2015-08-13 23:00:00,5,SilverStar
3,2015-08-14 04:00:00,2,SilverStar
4,2015-08-14 05:00:00,3,SilverStar
5,2015-08-14 06:00:00,2,SilverStar
6,2015-08-14 07:00:00,2,SilverStar


,Time,SnowDepth,Location
,<dttm>,<dbl>,<chr>
1,2018-12-10 23:00:00,27.7,Callaghan
2,2018-12-11 00:00:00,27.7,Callaghan
3,2018-12-11 01:00:00,27.5,Callaghan
4,2018-12-11 02:00:00,28.0,Callaghan
5,2018-12-11 03:00:00,28.2,Callaghan
6,2018-12-11 04:00:00,28.4,Callaghan


,Time,AirTempCelcius,Location
,<dttm>,<dbl>,<chr>
1,2018-09-10 18:00:00,3.0,SilverStar
2,2018-09-10 19:00:00,3.8,SilverStar
3,2018-09-10 20:00:00,5.7,SilverStar
4,2018-09-10 21:00:00,6.3,SilverStar
5,2018-09-10 22:00:00,6.2,SilverStar
6,2018-09-10 23:00:00,5.7,SilverStar


,Time,AirTempCelcius,Location
,<dttm>,<dbl>,<chr>
1,2018-10-31 18:00:00,3.2,Callaghan
2,2018-10-31 19:00:00,3.2,Callaghan
3,2018-10-31 20:00:00,3.5,Callaghan
4,2018-10-31 21:00:00,3.9,Callaghan
5,2018-10-31 22:00:00,4.1,Callaghan
6,2018-10-31 23:00:00,4.3,Callaghan


Next lets join all these data sets together. 

In [55]:
#Adding All the SnowDepth Data together
snow_data <- rbind(snow_depth_silverstar,snow_depth_callaghan)
#head(snow_data)
#Adding All the AirTemp Data together
temp_data <- rbind(air_temp_silverstar,air_temp_callaghan)
#head(temp_data)

#Adding all the telementary data together:
telementary <- merge(snow_data, temp_data)
head(telementary)



,Time,Location,SnowDepth,AirTempCelcius
,<dttm>,<chr>,<dbl>,<dbl>
1,2018-09-11 09:00:00,SilverStar,3,2.1
2,2018-09-12 14:00:00,SilverStar,0,1.8
3,2018-09-12 18:00:00,SilverStar,3,4.0
4,2018-09-13 15:00:00,SilverStar,1,1.1
5,2018-09-17 01:00:00,SilverStar,0,0.4
6,2018-09-18 17:00:00,SilverStar,3,2.2


As stated in our introduction, the seasons play a desicive role in snow depth and air temperature. Therefore we will only look at data from the month of January, which is conviently in the dead of the Canadian winter.

In [56]:
jan_data <- telementary |>
    filter(as.numeric(strftime(telementary$Time, "%m")) == 1)

head(jan_data)

,Time,Location,SnowDepth,AirTempCelcius
,<dttm>,<chr>,<dbl>,<dbl>
1,2019-01-01 00:00:00,Callaghan,155.1,-3.2
2,2019-01-01 00:00:00,SilverStar,152.0,-11.9
3,2019-01-01 01:00:00,Callaghan,154.5,-4.0
4,2019-01-01 01:00:00,SilverStar,152.0,-12.9
5,2019-01-01 02:00:00,Callaghan,154.2,-3.9
6,2019-01-01 02:00:00,SilverStar,152.0,-12.2


For curiousity sake lets graph the samples of snow depth at Silver Star that we have in 2019.

In [66]:
jan_data$Time <- lubridate::dmy(as.character(jan_data$Time))



winter2019_plot <- telementary |>
    select(Time, Location, SnowDepth) |>
    filter(Location == "SilverStar") |>
    ggplot() +
    geom_point(aes(x = format(telementary$Time, "%d"), y = SnowDepth))

# winter2019_plot

Time,Location,SnowDepth,AirTempCelcius
<date>,<chr>,<dbl>,<dbl>
NA,Callaghan,155.1,-3.2
NA,SilverStar,152.0,-11.9
NA,Callaghan,154.5,-4.0
NA,SilverStar,152.0,-12.9
NA,Callaghan,154.2,-3.9
NA,SilverStar,152.0,-12.2
NA,Callaghan,154.0,-3.9
NA,SilverStar,152.0,-11.8
NA,Callaghan,153.7,-4.3


## Methods: Plan

asdfasdfasdfasdfsadf

##### What do we expect to find?

- bullet point filler
- hi

##### What impact could such findings have?

- bullet point filler
- hi

##### What future questions could this lead to?

- bullet point filler
- hi

## References

Grünewald, T., Bühler, Y., & Lehning, M. (2014). Elevation dependency of mountain snow depth. The Cryosphere, 8(6), 2381–2394. https://doi.org/10.5194/tc-8-2381-2014

Kirchner, P. B., Bales, R. C., Molotch, N. P., Flanagan, J., & Guo, Q. (2014). LiDAR measurement of seasonal snow accumulation along an elevation gradient in the southern Sierra Nevada, California. Hydrology and Earth System Sciences, 18(10), 4261–4275. https://doi.org/10.5194/hess-18-4261-2014

Ye, H., & Ellison, M. (2003). Changes in transitional snowfall season length in northern Eurasia: TRANSITIONAL SNOWFALL SEASON. Geophysical Research Letters, 30(5), n/a-n/a. https://doi.org/10.1029/2003GL016873


# REMEMBER TO CITE DATA SETS